In [1]:
from selenium import webdriver
from collections import defaultdict

In [2]:
import time
import pandas as pd

# put your keyword here

In [3]:
keyword="data scientist"

# this is the linkedin url for job search

In [4]:
url = f"https://www.linkedin.com/jobs/search?keywords={keyword}&location=Indonesia&trk=public_jobs_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0"

# Open your webdriver and open the URL

In [5]:
wd = webdriver.Edge(executable_path="./msedgedriver.exe")
wd.get(url)

# get number of jobs

In [6]:
no_of_jobs = int(wd.find_element_by_css_selector("h1>span").get_attribute("innerText"))
print (no_of_jobs)

306


# Scroll all the job until LinkedIn shows all the jobs

In [7]:
i = 2
while i <= int(no_of_jobs/25)+1: 
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    try:
        wd.find_element_by_xpath('/html/body/main/div/section/button').click()
        time.sleep(5)
    except:
        pass
        time.sleep(5)

# get all the job inside a list

In [8]:
jobs_lists = wd.find_element_by_class_name('jobs-search__results-list')
jobs = jobs_lists.find_elements_by_tag_name('li') # return a list

# for every job, get all the information in this code [Job Id, Job Title, Company Name, Location, Date, Job_Link, Company_Link, Job Description,Seniority,Employment Type, Industries, and application numbers

In [17]:
job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []
company_link=[]
jd = []
job_pay = []
seniority = []
emp_type = []
job_func = defaultdict()
industries = defaultdict()
applicant_num=[]
for job in jobs[0:50]:
    job_id0 = job.get_attribute('data-id')
    job_id.append(job_id0)

    job_title0 = job.find_element_by_css_selector('h3').get_attribute('innerText')
    job_title.append(job_title0)

    company_name0 = job.find_element_by_css_selector('h4').get_attribute('innerText')
    company_name.append(company_name0)

    location0 = job.find_element_by_css_selector('[class="job-result-card__location"]').get_attribute('innerText')
    location.append(location0)

    date0 = job.find_element_by_css_selector('div>div>time').get_attribute('datetime')
    date.append(date0)

    job_links = job.find_elements_by_css_selector('a')
    try:
        company_link.append(job_links[1].get_attribute('href'))
    except:
        company_link.append("NA")
        
    job_link.append(job_links[0].get_attribute('href'))
    #for item in range(len(jobs)):
    
    # clicking job to view job details
    job_click_path = f'/html/body/main/div/section[2]/ul/li[{jobs.index(job)+1}]/img'
    job_click = job.find_element_by_xpath(job_click_path).click()
    time.sleep(5)
    
    try:
        applicant_path = '/html/body/main/section/div[2]/section[1]/div[1]//div[1]/h3[2]/figure/figcaption'
        applicant = job.find_element_by_xpath(applicant_path).get_attribute('innerText')
        
    except:
        try:
            #print(company_name0)
            applicant_path = '/html/body/main/section/div[2]/section[1]/div[1]/div[1]/h3[2]/span[2]'
            applicant = job.find_element_by_xpath(applicant_path).get_attribute('innerText')
        except:
            applicant_path = '/html/body/main/section/div[2]/section[1]/div[1]/div[1]/h3[3]/figure/figcaption'
            applicant = job.find_element_by_xpath(applicant_path).get_attribute('innerText')
        
    applicant_num.append(applicant)    

    jd_path = '/html/body/main/section/div[2]/section[2]/div'
    jd0 = job.find_element_by_xpath(jd_path).get_attribute('innerText')
    
    if "Base pay range" in jd0:
        order_=3
        jd_path = '/html/body/main/section/div[2]/section[3]/div'
        jd0 = job.find_element_by_xpath(jd_path).get_attribute('innerText')
        jd.append(jd0)
    else:
        jd.append(jd0)
        order_=2
    
    try:
        seniority_path = f'/html/body/main/section/div[2]/section[{order_}]/ul/li[1]/span'
        seniority0 = job.find_element_by_xpath(seniority_path).get_attribute('innerText')
        seniority.append(seniority0)
    except:
        print(company_name0)

    emp_type_path = f'/html/body/main/section/div[2]/section[{order_}]/ul/li[2]/span'
    emp_type0 = job.find_element_by_xpath(emp_type_path).get_attribute('innerText')
    emp_type.append(emp_type0)
    
    job_func0=[]
    industries0=[]
    job_func_path = f'/html/body/main/section/div[2]/section[{order_}]/ul/li[3]/span'
    job_func_elements = job.find_elements_by_xpath(job_func_path)
    for element in job_func_elements:
        job_func0.append(element.get_attribute('innerText'))
        job_func_final = ",".join(job_func0)
    job_func[job_id0]=(job_func_final)
    industries_path = '//*[@id="main-content"]/section/div[2]/section[2]/ul/li[4]/span'
    industries_elements = job.find_elements_by_xpath(industries_path)
    for element in industries_elements:
        industries0.append(element.get_attribute('innerText'))
        industries_final = ",".join(industries0)
        industries[job_id0]=(industries_final)

# create a dataframe from all the jobs

In [19]:
job_data = pd.DataFrame({'ID': job_id,
'Date': date,
'Company': company_name,
'Title': job_title,
'Location': location,
'Description': jd,
'Level': seniority,
'num_Applicant': applicant_num,                       
'Type': emp_type,
'Job_Link': job_link,
'Company_Link':company_link
                    
})
# cleaning description column
job_data['Description'] = job_data['Description'].str.replace('\n',' ')
#job_data.to_excel('LinkedIn Job Data_Data Scientist.xlsx', index = False)

In [20]:
job_data = job_data.set_index('ID')

# combine job data table with function table and industries table

In [21]:
Function_table = pd.DataFrame({"ID":job_func.keys(),"Function":job_func.values()}).set_index("ID")
industries_table = pd.DataFrame({"ID":industries.keys(),"Industries":industries.values()}).set_index("ID")

In [22]:
job_data_2 = job_data.join(Function_table).join(industries_table)

In [23]:
job_data_2

,Date,Company,Title,Location,Description,Level,num_Applicant,Type,Job_Link,Company_Link,Function,Industries
ID,,,,,,,,,,,,
2495155469,2021-03-31,Traveloka,Data Scientist,"Jakarta, Jakarta, Indonesia",Given a business problem and requirements from...,Entry level,46 applicants,Full-time,https://id.linkedin.com/jobs/view/data-scienti...,https://id.linkedin.com/company/traveloka-com?...,"Engineering,Information Technology",Internet
2466698262,2021-04-03,QuantumBlack,Data Scientist - QuantumBlack,"Jakarta, Jakarta, Indonesia",Apply Now Qualifications Bachelor or MSc or ...,Executive,42 applicants,Full-time,https://id.linkedin.com/jobs/view/data-scienti...,https://uk.linkedin.com/company/quantumblack?t...,"Consulting,Information Technology,Analyst","Information Technology and Services,Computer S..."
2479097487,2021-04-07,Tokopedia,Data Science Lead,"Jakarta, Indonesia",Job Description: Engaging with business teams...,Mid-Senior level,78 applicants,Full-time,https://id.linkedin.com/jobs/view/data-science...,https://id.linkedin.com/company/pt--tokopedia?...,"Analyst,Science",Internet
2479941826,2021-04-08,PT. XL Axiata Tbk,Sr. Data Scientist - CVM,"Jakarta, Indonesia",Roles : Lead the development of predictive cu...,Associate,90 applicants,Full-time,https://id.linkedin.com/jobs/view/sr-data-scie...,https://id.linkedin.com/company/pt.-xl-axiata-...,"Analyst,Marketing,Business Development","Telecommunications,Internet,Computer & Network..."
2463401542,2021-03-16,FNA,Data Scientist - Remote,"Jakarta, Jakarta, Indonesia",Title: Data Scientist Location: Remotely from...,Executive,112 applicants,Full-time,https://id.linkedin.com/jobs/view/data-scienti...,https://uk.linkedin.com/company/fnalab?trk=pub...,"Engineering,Information Technology","Information Technology and Services,Computer S..."
2479007184,2021-04-07,tiket.com,Data Scientist Lead,"Jakarta, Indonesia",Job Overview The ideal candidate is adept ...,Mid-Senior level,35 applicants,Full-time,https://id.linkedin.com/jobs/view/data-scienti...,https://id.linkedin.com/company/pt-global-tike...,"Finance,Accounting/Auditing,Administrative",Information Technology and Services
2482805404,2021-04-09,PT DJARUM,AI Engineer,"Jakarta, Indonesia",Our Growing Technology company is looking for ...,Entry level,151 applicants,Full-time,https://id.linkedin.com/jobs/view/ai-engineer-...,https://id.linkedin.com/company/pt-djarum?trk=...,Information Technology,Information Technology and Services
2404196558,2021-02-01,Sayurbox,Data Scientist,"Pasar Minggu, Jakarta, Indonesia",Sayurbox is a provider of fresh produce to hou...,Mid-Senior level,131 applicants,Full-time,https://id.linkedin.com/jobs/view/data-scienti...,https://id.linkedin.com/company/sayurboxoffici...,"Engineering,Information Technology","Marketing and Advertising,Internet,Consumer Goods"
2411127378,2021-03-28,Ajaib,Head of Data Science,"Jakarta, Indonesia",Direct message the job poster from Ajaib Ande...,Mid-Senior level,Over 200 applicants,Full-time,https://id.linkedin.com/jobs/view/head-of-data...,https://id.linkedin.com/company/ajaib-id?trk=p...,Internet,NaN


In [24]:
job_data_2.to_excel(f"Job_data_{keyword}.xlsx")